<a href="https://colab.research.google.com/github/joseluisgch/Visor-de-Accesibilidad/blob/main/Tiempo_de_desplazamiento_en_ciudades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Visor de accesibilidad mediante OSM

In [5]:
!pip install openrouteservice folium ipywidgets ipyleaflet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.4 MB/s eta 0:00:00


In [9]:
# Instalar ipywidgets
!pip install ipywidgets

# Instalar openrouteservice, folium, shapely y otras dependencias
!pip install openrouteservice folium shapely ipyleaflet

# Habilitar widgets en Colab
!jupyter nbextension enable --py widgetsnbextension --sys-prefix


Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/usr/etc/jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/usr/etc/jupyter/nbconfig/notebook.json


In [19]:
import openrouteservice
from ipyleaflet import Map, Marker, GeoJSON, LayersControl, FullScreenControl, WidgetControl
from ipywidgets import interact, widgets
import json

# Configurar la clave de API de OpenRouteService
API_KEY = '5b3ce3597851110001cf6248df1c46e4a40f4150a1f9c089c12d69f2'
cliente_ors = openrouteservice.Client(key=API_KEY)

# Coordenadas iniciales para Lima
coordenadas_iniciales = [-77.0428, -12.0464]  # Longitud, Latitud

# Crear mapa interactivo
mapa_interactivo = Map(center=(coordenadas_iniciales[1], coordenadas_iniciales[0]), zoom=12)
mapa_interactivo.add_control(FullScreenControl())
mapa_interactivo.add_control(LayersControl(position="topright"))

# Agregar marcador inicial
marcador = Marker(location=(coordenadas_iniciales[1], coordenadas_iniciales[0]), draggable=True)
mapa_interactivo.add_layer(marcador)

# Función para calcular isocronas ajustando velocidades para tráfico alto
def calcular_isocronas(origen, modo, tiempo_minutos):
    # Simulación de velocidades más lentas para tráfico
    if modo == "driving-car":
        tiempo_minutos = tiempo_minutos * 0.35  # Reducimos el alcance en un 65%

    parametros = {
        "locations": [origen],
        "profile": modo,
        "range": [int(tiempo_minutos * 60)],  # Tiempo en segundos
        "range_type": "time",  # Tipo de rango basado en tiempo
    }
    isocronas = cliente_ors.isochrones(**parametros)
    return isocronas

# Función para actualizar el mapa con isocronas
def actualizar_isocrona(tiempo_minutos, modo_transporte):
    # Eliminar isocronas anteriores
    for layer in list(mapa_interactivo.layers):
        if isinstance(layer, GeoJSON):
            mapa_interactivo.remove_layer(layer)

    # Calcular isocronas con la ubicación actual del marcador
    origen = [marcador.location[1], marcador.location[0]]  # Latitud, Longitud
    isocronas = calcular_isocronas(origen, modo_transporte, tiempo_minutos)

    # Dibujar isocronas en el mapa
    for feature in isocronas['features']:
        geo_json = GeoJSON(
            data=feature,
            style={
                "color": "red" if modo_transporte == "driving-car" else "blue",
                "opacity": 0.5,
                "weight": 2,
                "fillColor": "red" if modo_transporte == "driving-car" else "blue",
                "fillOpacity": 0.3,
            },
        )
        mapa_interactivo.add_layer(geo_json)

# Widgets para tiempo y modo de transporte
slider_tiempo = widgets.IntSlider(
    min=5, max=60, step=5, value=15, description="Tiempo (min)"
)
dropdown_modo = widgets.Dropdown(
    options={
        "A pie": "foot-walking",
        "Bicicleta": "cycling-regular",
        "Automóvil (Hora punta)": "driving-car",
    },
    value="foot-walking",
    description="Modo",
)

# Función interactiva para actualizar el mapa dinámicamente
def actualizar_mapa(tiempo, modo):
    actualizar_isocrona(tiempo, modo)

# Conectar widgets con la función interactiva
widgets_interactivos = widgets.VBox([slider_tiempo, dropdown_modo])
control_widgets = WidgetControl(widget=widgets_interactivos, position="topright")
mapa_interactivo.add_control(control_widgets)

interact(actualizar_mapa, tiempo=slider_tiempo, modo=dropdown_modo)

# Mostrar mapa interactivo
mapa_interactivo


interactive(children=(IntSlider(value=15, description='Tiempo (min)', max=60, min=5, step=5), Dropdown(descrip…

Map(center=[-12.0464, -77.0428], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…